In [24]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from _version import __version__
from data import *
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# Create a SweepExperiment
experiment = SweepExperiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)

In [26]:
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments/experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()


In [27]:
# get the completed experiments with code version 5
df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==__version__)]

,experiment_id,code_version,state_evolution_repetitions,erm_repetitions,date,alphas,epsilons,lambdas,tau,d,completed
57,efd6ee50-96f6-4ddd-9a84-ce6c74469637,5,2023-03-23T16:02:50.530681,1,1,"[0.6444444444444445, 2.277777777777778, 5.0]","[0.0, 0.02]",[0.001],2.0,300,1


In [28]:
df_state_evolution[df_state_evolution["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]

,id,code_version,experiment_id,date,sigma,q,m,initial_condition,alpha,epsilon,tau,lam,abs_tol,min_iter,max_iter,blend_fpe,int_lims,generalization_error
1319,8f503f98-1ffd-4c42-93d5-a2570dbfe451,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:05:32.308948,0.000470,0.031941,0.178208,"[0.5, 0.5, 0.5]",5.000000,0.00,2.0,0.001,0.0001,20,5000,0.75,20.0,0.045324
1320,0996337b-3089-439b-9ef5-31d1092311f4,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:05:33.916363,0.001069,0.032918,0.180310,"[0.5, 0.5, 0.5]",2.277778,0.00,2.0,0.001,0.0001,20,5000,0.75,20.0,0.066818
1321,21f0d240-5d12-4152-ab27-cc0ed8f5bcd2,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:05:34.684284,0.000475,0.032714,0.180356,"[0.5, 0.5, 0.5]",5.000000,0.02,2.0,0.001,0.0001,20,5000,0.75,20.0,0.045062
1322,37eb4875-b910-4759-b57d-601f54c84df2,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:05:36.635780,0.001081,0.033722,0.182507,"[0.5, 0.5, 0.5]",2.277778,0.02,2.0,0.001,0.0001,20,5000,0.75,20.0,0.066441
1323,2247b69c-f913-4e34-a420-e1e3a28ae15f,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:05:45.660237,0.004508,0.037626,0.189979,"[0.5, 0.5, 0.5]",0.644444,0.00,2.0,0.001,0.0001,20,5000,0.75,20.0,0.123619
1324,6c4cca60-bcf8-448b-b2b3-e7c9f9fc7ef9,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:05:47.592770,0.004567,0.038593,0.192433,"[0.5, 0.5, 0.5]",0.644444,0.02,2.0,0.001,0.0001,20,5000,0.75,20.0,0.122996


In [30]:
df_erm

,id,code_version,experimend_id,date,q,m,rho,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,chosen_minimizer,training_error,test_loss,d,tau,alpha
0,32cf4904-358b-42d7-9f25-98b0813f3cce,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:02:51.674717,19.922587,0.825360,0.988627,0.185975,0.02,0.001,0.46955,0.440456,gd,0.000000,2.599432,300,2.0,0.643333
1,cb85e769-5fc0-48a3-b28b-c4cce62f9a27,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:02:51.689798,70.444430,2.132845,0.967957,0.258290,0.00,0.001,0.46935,0.416841,gd,0.000000,3.077289,300,2.0,0.643333
2,4f186bb6-da89-4f6b-a4df-b8d8fb4cea80,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:02:51.766877,1.332306,0.490223,0.896765,0.448490,0.02,0.001,0.44400,0.351962,gd,0.180088,0.957954,300,2.0,2.276667
3,54edd0c5-e1f8-440a-a9ec-a76f7ac5bd2f,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:02:53.048897,2.700668,0.920617,0.910272,0.587162,0.00,0.001,0.41380,0.300245,gd,0.263333,0.784465,300,2.0,5.000000
4,27dea92f-9560-4279-b507-dff7f1e87bde,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:02:54.387506,0.302610,0.320349,0.884501,0.619202,0.02,0.001,0.41235,0.287456,gd,0.262000,0.764385,300,2.0,5.000000
5,4861cb8a-0edc-46d5-8036-69c2a8d4d26b,5,efd6ee50-96f6-4ddd-9a84-ce6c74469637,2023-03-23T16:02:58.382009,157.831368,4.243541,0.943821,0.347686,0.00,0.001,0.44595,0.386968,gd,0.064422,4.066541,300,2.0,2.276667


In [36]:
# rename experimend_id to experiment_id
df_erm = df_erm.rename(columns={"experimend_id":"experiment_id"})

In [ ]:
erm = df_erm[df_erm["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]

In [48]:
# merge df_erm and df_state_evolution on experiment id efd6ee50-96f6-4ddd-9a84-ce6c74469637
erm = df_erm[df_erm["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
# print the number of rows in each dataframe
print("erm: %d, state_evolution: %d" % (erm.shape[0],state_evolution.shape[0]))
# round alpha and epsilon to 2 decimals
erm["alpha"] = erm["alpha"].round(2)
erm["epsilon"] = erm["epsilon"].round(2)
state_evolution["alpha"] = state_evolution["alpha"].round(2)
state_evolution["epsilon"] = state_evolution["epsilon"].round(2)
# keep alpha, epsilon and the generalization errors
erm = erm[["alpha","epsilon","generalization_error_erm","generalization_error_overlap"]]
state_evolution = state_evolution[["alpha","epsilon","generalization_error"]]

# join the dataframes on alpha an epsilon and add suffixes to the columns
df = erm.merge(state_evolution, on=["alpha","epsilon"], suffixes=("_erm","_state_evolution"))
# print the number of rows in the joined dataframe
print("joined: %d" % df.shape[0])
df

erm: 6, state_evolution: 6
joined: 6


/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["alpha"] = state_evolution["alpha"].round(2)
/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["epsilon"] = state_evolution["epsilon"].round(2)


,alpha,epsilon,generalization_error_erm,generalization_error_overlap,generalization_error
0,0.64,0.02,0.46955,0.440456,0.122996
1,0.64,0.00,0.46935,0.416841,0.123619
2,2.28,0.02,0.44400,0.351962,0.066441
3,5.00,0.00,0.41380,0.300245,0.045324
4,5.00,0.02,0.41235,0.287456,0.045062
5,2.28,0.00,0.44595,0.386968,0.066818
